In [1]:
import re

In [2]:
with open("post.txt", 'r') as f:
    post_text = f.read()

In [3]:
bb_uid = '17hlnlju'

In [4]:
paired_tags = 'quote|b|u|i|list|url|img|\*'
start_pattern = f'\[((?:{paired_tags})[^\:]*)\:{bb_uid}\]'
end_pattern = f'\[\/({paired_tags})(?:\:m|\:u|\:o)?\:{bb_uid}\]'
split1 = re.split(start_pattern, post_text)
x = [
        [ {"type" : "text", "value" : e } if i%2 ==0 else {"type" : "finish", "value" : e} for i, e in enumerate(re.split(end_pattern, item))]
        if index%2 == 0
        else [{"type" : "start", "value" : item}]
        for index, item in enumerate(split1)
    ]
t = []
for i in x:
    t += i

In [5]:
t.reverse()

root = {
    "type" : "root",
    "elements" : []
}
branch = root
breadcrumbs = []

while len(t) > 0:
    item = t.pop()

    if item['type'] == 'text':
        branch['elements'].append({
            "type" : "text",
            "value" : item['value']
        })
    
    if item['type'] == 'start':
        item_parts =  item['value'].split('=')
        item_parts.append('')
        leaf = {
            "type" : item_parts[0],
            "value" : item_parts[1],
            "elements" : []
        }
        branch['elements'].append(leaf)
        breadcrumbs.append(branch)
        branch = leaf

    if item['type'] == 'finish':
        branch = breadcrumbs.pop()

root

{'type': 'root',
 'elements': [{'type': 'text',
   'value': '<!-- s:fight: --><img src="{SMILIES_PATH}/bije.gif" alt=":fight:" title="fight" /><!-- s:fight: -->  <!-- s:hurra: --><img src="{SMILIES_PATH}/hura.gif" alt=":hurra:" title="Hurra!" /><!-- s:hurra: -->  <!-- s:olaboga: --><img src="{SMILIES_PATH}/olaboga.gif" alt=":olaboga:" title="Olaboga" /><!-- s:olaboga: -->  <!-- s:wnerw: --><img src="{SMILIES_PATH}/wnerw.gif" alt=":wnerw:" title="wnerw" /><!-- s:wnerw: --> \n\n <!-- s:fight: --><img src="{SMILIES_PATH}/bije.gif" alt=":fight:" title="fight" /><!-- s:fight: -->  <!-- s:hurra: --><img src="{SMILIES_PATH}/hura.gif" alt=":hurra:" title="Hurra!" /><!-- s:hurra: -->  <!-- s:olaboga: --><img src="{SMILIES_PATH}/olaboga.gif" alt=":olaboga:" title="Olaboga" /><!-- s:olaboga: -->  <!-- s:wnerw: --><img src="{SMILIES_PATH}/wnerw.gif" alt=":wnerw:" title="wnerw" /><!-- s:wnerw: --> \n\n'},
  {'type': 'quote',
   'value': '&quot;trekker&quot;',
   'elements': [{'type': 'text', 'value

In [6]:
def render(element):
    if element['type'] = "text":
        return element['value']
    if element['type'] = "b":
        t = ' **'
        for i in element['elements']:
            t += render(i)
        t += '** '
    return t   
        

SyntaxError: cannot assign to subscript here. Maybe you meant '==' instead of '='? (1246246279.py, line 2)

In [364]:
start_pattern = f'\[quote(?:=\&quot\;([^\&]+)\&quot\;)?:{bb_uid}\]'
end_pattern = f'\[(\/)quote:{bb_uid}\]'
tag_pattern =  start_pattern + '|' + end_pattern

In [365]:
quote_parts = re.split(tag_pattern, post_text)

In [366]:
quote_parts.reverse()
tree = []
level = 0
tree.append({
    "level" : level,
    "author" : '',
    "text" : quote_parts.pop()
})
while len(quote_parts) > 0:
    
    tag_start = quote_parts.pop()
    tag_end = quote_parts.pop()
    tag_txt = quote_parts.pop()
    
    auth = ''

    if tag_end == '/':
        level -= 1    
    else:
        level += 1
        if tag_start is not None:
            auth = tag_start
    
    tree.append({
        "level" : level,
        "author" : auth,
        "text" : tag_txt
    })
    

tree

[{'level': 0,
  'author': '',
  'text': 'bold\n\n[b:2dgkza2d]lorem[/b:2dgkza2d]\n\nitalic\n\n[i:2dgkza2d]lorem[/i:2dgkza2d]\n\nunderscore\n\n[u:2dgkza2d]lorem[/u:2dgkza2d]\n\ncytaty\n'},
 {'level': 1, 'author': '', 'text': '\njeden\n'},
 {'level': 2, 'author': 'trekker', 'text': 'jeden.dwa'},
 {'level': 1, 'author': '', 'text': '\ndwa\n'},
 {'level': 2, 'author': '', 'text': 'dwa.dwa'},
 {'level': 1, 'author': '', 'text': '\ntrzy\n'},
 {'level': 0, 'author': '', 'text': '\n\n'},
 {'level': 1, 'author': 'beniu', 'text': 'cztery'},
 {'level': 0,
  'author': '',
  'text': '\n\nkod\n\n[code:2dgkza2d]lorem[/code:2dgkza2d]\n\nlisty\n\n[list:2dgkza2d]\n[*:2dgkza2d]aaa[/*:m:2dgkza2d]\n[*:2dgkza2d]bbb[/*:m:2dgkza2d][/list:u:2dgkza2d]\n\n[list=1:2dgkza2d]\n[*:2dgkza2d]ccc[/*:m:2dgkza2d]\n[*:2dgkza2d]ddd[/*:m:2dgkza2d][/list:o:2dgkza2d]\n\n[list=1:2dgkza2d]\n[*:2dgkza2d]eee[/*:m:2dgkza2d]\n[*:2dgkza2d]fff\n[list=a:2dgkza2d]\n[*:2dgkza2d]ggg[/*:m:2dgkza2d]\n[*:2dgkza2d]hhh[/*:m:2dgkza2d][/list:o:2

In [367]:
tree

[{'level': 0,
  'author': '',
  'text': 'bold\n\n[b:2dgkza2d]lorem[/b:2dgkza2d]\n\nitalic\n\n[i:2dgkza2d]lorem[/i:2dgkza2d]\n\nunderscore\n\n[u:2dgkza2d]lorem[/u:2dgkza2d]\n\ncytaty\n'},
 {'level': 1, 'author': '', 'text': '\njeden\n'},
 {'level': 2, 'author': 'trekker', 'text': 'jeden.dwa'},
 {'level': 1, 'author': '', 'text': '\ndwa\n'},
 {'level': 2, 'author': '', 'text': 'dwa.dwa'},
 {'level': 1, 'author': '', 'text': '\ntrzy\n'},
 {'level': 0, 'author': '', 'text': '\n\n'},
 {'level': 1, 'author': 'beniu', 'text': 'cztery'},
 {'level': 0,
  'author': '',
  'text': '\n\nkod\n\n[code:2dgkza2d]lorem[/code:2dgkza2d]\n\nlisty\n\n[list:2dgkza2d]\n[*:2dgkza2d]aaa[/*:m:2dgkza2d]\n[*:2dgkza2d]bbb[/*:m:2dgkza2d][/list:u:2dgkza2d]\n\n[list=1:2dgkza2d]\n[*:2dgkza2d]ccc[/*:m:2dgkza2d]\n[*:2dgkza2d]ddd[/*:m:2dgkza2d][/list:o:2dgkza2d]\n\n[list=1:2dgkza2d]\n[*:2dgkza2d]eee[/*:m:2dgkza2d]\n[*:2dgkza2d]fff\n[list=a:2dgkza2d]\n[*:2dgkza2d]ggg[/*:m:2dgkza2d]\n[*:2dgkza2d]hhh[/*:m:2dgkza2d][/list:o:2

In [368]:
lines = []
for item in tree:
    a = item['author']
    l = item['level']
    t = item['text']
    if a != '':
        lines.append((l - 1) * '> ' + f'({a} napisał/a:)')
    for line in t.split('\n'):
        lines.append(l * '>' + ' ' + line)

In [369]:
'aaa'.replace(old, new)

NameError: name 'old' is not defined

x<em>2</em>

In [371]:
lines = [l.replace(f'[b:{bb_uid}]', ' **') for l in lines]
lines = [l.replace(f'[/b:{bb_uid}]', '** ') for l in lines]
lines = [l.replace(f'[u:{bb_uid}]', ' <u>') for l in lines]
lines = [l.replace(f'[/u:{bb_uid}]', '</u> ') for l in lines]
lines = [l.replace(f'[i:{bb_uid}]', ' _') for l in lines]
lines = [l.replace(f'[/i:{bb_uid}]', '_ ') for l in lines]
for l in lines:
    print(l)

 bold
 
  **lorem** 
 
 italic
 
  _lorem_ 
 
 underscore
 
  <u>lorem</u> 
 
 cytaty
 
> 
> jeden
> 
> (trekker napisał/a:)
>> jeden.dwa
> 
> dwa
> 
>> dwa.dwa
> 
> trzy
> 
 
 
 
(beniu napisał/a:)
> cztery
 
 
 kod
 
 [code:2dgkza2d]lorem[/code:2dgkza2d]
 
 listy
 
 [list:2dgkza2d]
 [*:2dgkza2d]aaa[/*:m:2dgkza2d]
 [*:2dgkza2d]bbb[/*:m:2dgkza2d][/list:u:2dgkza2d]
 
 [list=1:2dgkza2d]
 [*:2dgkza2d]ccc[/*:m:2dgkza2d]
 [*:2dgkza2d]ddd[/*:m:2dgkza2d][/list:o:2dgkza2d]
 
 [list=1:2dgkza2d]
 [*:2dgkza2d]eee[/*:m:2dgkza2d]
 [*:2dgkza2d]fff
 [list=a:2dgkza2d]
 [*:2dgkza2d]ggg[/*:m:2dgkza2d]
 [*:2dgkza2d]hhh[/*:m:2dgkza2d][/list:o:2dgkza2d][/*:m:2dgkza2d]
 [*:2dgkza2d]iii[/*:m:2dgkza2d][/list:o:2dgkza2d]
 
 [img:2dgkza2d]https&#58;//v&#46;wpimg&#46;pl/MDc0OC5wYiUCUjlwGgxvMEEKbSpcVWFmFhJ1YRpFfHwbAXxzGgQqMxMeODNWEyNqE1xgIgdHdCZWBXl1DUYvJloELnZTRSl9U1N3IQVDfHRUBHdtRRkqZh4[/img:2dgkza2d]
 
 [youtube:2dgkza2d]iBxVRwynmmE[/youtube:2dgkza2d]
 
 [url:2dgkza2d]http&#58;//www&#46;wp&#46;pl[/url:2dgkza2d

In [424]:
items_list = [
    {"l" : 0, "v" : 'a', "type" : 'text'},
    {"l" : 1, "v" : 'b', "type" : 'b'},
    {"l" : 1, "v" : 'c', "type" : 'quote'},
    {"l" : 2, "v" : 'd', "type" : 'quote'},
    {"l" : 1, "v" : 'e', "type" : 'text'},
    {"l" : 2, "v" : 'f'},
    {"l" : 1, "v" : 'g'},
    {"l" : 0, "v" : 'h'},
    {"l" : 1, "v" : 'i'},
    {"l" : 0, "v" : 'j'}
]
items_list.reverse()

In [425]:
root = {
    "type" : 'text',
    "content" : []
}
branch = root
levels = []
current_level = 0

while len(items_list) > 0:
    item = items_list.pop()
    
    if item['l'] == current_level:
        branch['content'].append(
            {
                "type" : 'text',
                "content" : [item['v']]
            }
        )

    if item['l'] > current_level:
        current_level += 1
        
        new_branch = {
            "type" : "text",
            "content" : []
        }
        new_branch['content'].append(item['v'])
        
        branch['content'].append(new_branch)
        levels.append(branch)
        branch = new_branch
    
    if item['l'] < current_level:
        current_level -= 1
        branch = levels.pop()
        branch['content'].append(item['v'])


root


{'type': 'text',
 'content': ['a',
  {'type': 'text',
   'content': ['b',
    'c',
    {'type': 'text', 'content': ['d']},
    'e',
    {'type': 'text', 'content': ['f']},
    'g']},
  'h',
  {'type': 'text', 'content': ['i']},
  'j']}